# Clasificación de texto con vectores de frecuencias

[Pablo Carballeira] Partes de este código han sido adaptadas del código correspondiente a la especialización Natural Language Processing de DeepLearning.AI, y este [tutorial](https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk)

Puedes encontrar información sobre cómo trabajar en Colab aquí (https://colab.research.google.com/notebooks/intro.ipynb):





## Setup

Importamos y descargamos algunas librerías

In [ ]:
import nltk                                  # Python library for NLP
from nltk.corpus import twitter_samples      # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt              # visualization library
import numpy as np                           # library for scientific computing and matrix operations
import pandas as pd                          # Library for Dataframes 

from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('twitter_samples')

En este notebook nos centraremos en representar los tweets pre-procesados del notebook anterior como vectores matemáticos, sobre los que construir un clasificador de texto (clasificador de opinión). El ejemplo aquí representado es sencillo, pero ayuda a entender como convertir el texto en vectores numéricos  sobre los que se pueden aplicar algoritmos de aprendizaje automático estándar. 

Construiremos una función `build_freqs()` que creará un diccionario donde podemos buscar cuántas veces aparece una palabra del vocabulario en las listas de tweets positivos o negativos. Esto nos permite representar cada tweet con un vector de características que nos permitirá clasificarlo como positivo o negativo. Analizaremos si este tipo de descriptores son útiles para entrenar un clasificador de texto de manera eficiente. Finalmente, usaremos el dataset de Twitter para entrenar un clasificador real




## Función de pre-procesado
Utilizaremos la funcion  `process_tweet()` del notebook anterior para pre-procesar el texto de los tweets de forma adecuada


In [ ]:
# download the stopwords for the process_tweet function
nltk.download('stopwords')

import re
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer


def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

## Twitter dataset

Utilizamos el mismo dataset de tweets del notebook anterior: [Twitter dataset](http://www.nltk.org/howto/twitter.html#Using-a-Tweet-Corpus).

El dataset se separa en tweets positivos y negativos. Contiene 5000 tweets positivos y 5000 tweets negativos exactamente. La coincidencia exacta entre el número de muestras en cada clase no es una coincidencia. La intención es tener un conjunto de datos equilibrado. Eso no refleja las distribuciones reales de clases positivas y negativas en Twitter. Los conjuntos de datos equilibrados simplifican el diseño de la mayoría de los métodos de clasificación de textos. Sin embargo, es importante ser consciente de que este equilibrio de clases es artificial. 


In [ ]:
# select the lists of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

# concatenate the lists, 1st part is the positive tweets followed by the negative
tweets = all_positive_tweets + all_negative_tweets

# let's see how many tweets we have
print("Number of tweets: ", len(tweets))

Etiquetamos los tweeets con 1's para tweets positivos y 0's para tweets negativos. Fijémonos que la concatenación responde a la estructura de la variable `tweets` definida arriba, donde los primeros 5000 tweets son positivos y los siguientes 5000 tweets son negativos.

Podemos utilizar la librería de `numpy` para crear vectores de 0s y 1s facilmente:

* `np.ones()` - create an array of 1's
* `np.zeros()` - create an array of 0's
* `np.append()` - concatenate arrays

### Ejercicio
Crea la variable labels que contenga las etiquetas asociadas a los 10000 tweets

In [ ]:
# make a numpy array representing labels of the tweets
# labels = ???

## Función **build_freqs()**

Construyamos la función **build_freqs()** que utilizaremos para calcular los vectores de frecuencias de cada palabra y cada tuit.

Esta función debe crear un  diccionario que contiene, para cada palabra, las frecuencias (conteo absoluto) de aparición de esa palabra en los tweets positivos y negativos pertenecientes al corpus (conjunto total de tuits). Es decir cuenta, con qué frecuencia una palabra  se asoció con una etiqueta positiva '1' o una etiqueta negativa '0'.



Cada clave del diccionario `freqs` debe ser una tupla de 2 elementos que contiene una pareja `(word, y)`. `word` es un token de un tuit procesado, mientras que `y` es un número entero que representa la etiqueta: `1` para los tuits positivos y `0` para los tuits negativos. 

El valor asociado a esta clave es el número de veces que esa palabra aparece en el corpus  de tweets asociado a la etiqueta correspondiente

Por ejemplo:
``` 
# "followfriday" appears 25 times in the positive tweets
('followfriday', 1.0): 25

# "shame" appears 19 times in the negative tweets
('shame', 0.0): 19 
```

### Ejercicio

Completa la función, procesando el tweet, y aumentando la frecuencia correspondiente, para cada token del tuit. 

Ten en cuenta que cada palabra (token) puede estar o no presente en el diccionario. Si lo está debes incrementar el conteo. Si no, crear una nueva entrada en el dicionario.

In [ ]:
def build_freqs(tweets, labels):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        labels: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary that maps each key (word, sentiment) to its
        frequency value
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    labels_list = np.squeeze(labels).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}

    for y, tweet in zip(labels_list, tweets):
        # ???

    return freqs


## Frecuencias por palabra (tweets positivos y negativos)


Ahora, usamos el diccionario devuelto por la función `build_freqs()` para calcular las frecuencias de cada palabra del vocabulario en los tweets positivos y negativos en todo el corpus de tweets.

In [ ]:
# create frequency dictionary
freqs = build_freqs(tweets, labels)

# check data type
print(f'type(freqs) = {type(freqs)}')

# check length of the dictionary
print("Número de palabras en el vocabulario: ", len(freqs))

A continuación mostramos algunos ejemplos de las frecuencias calculadas para algunas palabras representativas. Fíjate que usamos las versiones normalizadas (radicalizadas) de cada término. 

Guardamos los vectores en la variable data para usarlos mas adelante en una representación gráfica.

In [ ]:
# select some words to appear in the report. we will assume that each word is unique (i.e. no duplicates)
keys = ['happi', 'merri', 'nice', 'good', 'bad', 'sad', 'mad', 'best', 'pretti',
        '❤', ':)', ':(', '😒', '😬', '😄', '😍', '♛',
        'song', 'idea', 'power', 'play', 'magnific',  'guitar', 'tv', 'walk']

# list representing our table of word counts.
# each element consist of a sublist with this pattern: [<word>, <positive_count>, <negative_count>]
data = []

# loop through our selected words
for word in keys:
    
    # initialize positive and negative counts
    pos = 0
    neg = 0
    
    # retrieve number of positive counts
    if (word, 1) in freqs:
        pos = freqs[(word, 1)]
        
    # retrieve number of negative counts
    if (word, 0) in freqs:
        neg = freqs[(word, 0)]
        
    # append the word counts to the table
    data.append([word, pos, neg])

print("  word","  pos", "  neg")
data

Fíjate como hay palabras claramente asociadas con tweets positivos, otras claramente asociadas con tweets negativos, y algunas neutras

### Ejercicio
Prueba con otras palabras. Puedes buscar opciones en la siguiente lista de palabras y añadirlas a la lista anterior


In [ ]:
print(freqs)

## Representación gráfica de las frecuencias de cada palabra

Podemos usar un diagrama de dispersión para inspeccionar estas frecuencias de forma gráfica. 

Representaremos las frecuencias en  escala logarítmica para reducir el rango  entre las frecuencias de algunas palabras (por ejemplo, `':)'` tiene 3568 conteos positivos mientras que solo 2 negativos).


In [ ]:
fig, ax = plt.subplots(figsize = (8, 8))

# convert positive raw counts to logarithmic scale. we add 1 to avoid log(0)
x = np.log([x[1] + 1 for x in data])  

# do the same for the negative counts
y = np.log([x[2] + 1 for x in data]) 

# Plot a dot for each pair of words
ax.scatter(x, y)  

# assign axis labels
plt.xlabel("Log Positive count")
plt.ylabel("Log Negative count")

# Add the word as the label at the same position as you added the points just before
for i in range(0, len(data)):
    ax.annotate(data[i][0], (x[i], y[i]), fontsize=12)

ax.plot([0, 9], [0, 9], color = 'red') # Plot the red line that divides the 2 areas.
plt.show()

La línea roja marca el límite entre las áreas positivas y negativas. Las palabras cercanas a la línea roja se pueden clasificar como neutras.

Este gráfico es fácil de interpretar. Muestra que los emoticonos `:)` y `:(` son muy importantes para el análisis de sentimientos. Por lo tanto no debemos permitir que los pasos de pre-procesamiento eliminen estos símbolos.

Si has añadido términos a la lista anterior. Es posible que haya ciertas sorpresas. Palabras que semánticamente son neutras pueden tener un caracter positivo/negativo fuerte. Esto se debe a una distribución artificialmente  desbalanceada, debido a que el dataset es de tamaño limitado.

## Representacion de tuits con vectores

Al igual que hemos hecho con las palabras, podemos representar cada tweet con un vector de frecuencias. En este caso, cada tuit estará representado por la suma de las frecuencias positivas y negativas de cada una de las palabras que contiene.

Por ejemplo, el tuit 'happy song' se compone de los tokens `['happi','song']`, cuyas frecuencias podemos comprobar más arriba. Las frecuencias positiva y negativa del tuit serán las siguientes: 

``` 
['happi','song']
pos: 211 + 22 = 233
neg: 25 + 27 = 52
``` 


La siguiente función debe sumar, por cada palabra del tweet su contribucion a la frecuencia positiva, y a frecuencia negativa

### Ejercicio

Completa la siguiente función, recorriendo los tokens de cada tuit, e incrementando los conteos positivo (x[0,0]) y negativo (x[0,1]) de forma adecuada, usando el vector de frecuencias de cada palabra.

Ten en cuenta que puede que una palabra no exista en el diccionario. En ese caso la función [get()](https://python-reference.readthedocs.io/en/latest/docs/dict/get.html), que puedes utilizar, debe devolver el valor 0

In [ ]:
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,2)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 2)) 
    
    # loop through each word in the list of words
    # ???
 
    assert(x.shape == (1, 2))
    return x

Vamos a visualizar los vectores de frecuencias de algunos tuits del dataset

In [ ]:
import random

# creating a list of column names
column_values = ['tweet', 'pos', 'neg', 'sentiment']

df = pd.DataFrame(columns=column_values)
for i in range(20):
    sample = random.randint(0,10000)
    tweet = tweets[sample]
    vector = extract_features(tweet, freqs)
    df.loc[i] = [ tweet, vector[0][0], vector[0][1],labels[sample]]

df.head(20)

# Visualizando los vectores descriptores de los tweets

En esta sección vamos a visualizar gráficamente los vectores de características que utilizamos como descriptores para nuestro corpus de tweets, y analizar si contienen información relevante para entrenar un clasificador efecivo (por ejemplo un regresor logístico).

Cargamos los vectores de características que tenemos pre-calculados en un CSV para todo nuestro corpus de tweets

In [ ]:
!mkdir /content/data
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1950LLLR4jwvi-4o1sOnGiWxyz2xB0cSn' -O data/logistic_features.csv

#https://drive.google.com/file/d/1950LLLR4jwvi-4o1sOnGiWxyz2xB0cSn/view?usp=sharing

Vamos a mostrar en una gráfica de dispersión los vectores de frecuencias correspondientes a cada uno de los tweets de nuestro corpus.  Además, coloreamos cada tweet, dependiendo de su clase. Los tweets positivos serán verdes y los tweets negativos serán rojos.


En esa misma gráfica comprobamos que ambas clases (positiva y negativa) son separables y que se puede entrenar un modelo sencillo para clasificar los tweets con una tasa de acierto muy elevada. La recta representa un plano que divide nuestro espacio de características en dos partes. Las muestras ubicadas por encima de ese plano se consideran positivas y las muestras ubicadas por debajo de ese plano se consideran negativas.














In [ ]:
theta = [7e-08, 0.0005239, -0.00055517]

# Equation for the separation plane
# It give a value in the negative axe as a function of a positive value
# f(pos, neg, W) = w0 + w1 * pos + w2 * neg = 0
# s(pos, W) = (w0 - w1 * pos) / w2
def neg(theta, pos):
    return (-theta[0] - pos * theta[1]) / theta[2]

# Equation for the direction of the sentiments change
# We don't care about the magnitude of the change. We are only interested 
# in the direction. So this direction is just a perpendicular function to the 
# separation plane
# df(pos, W) = pos * w2 / w1
def direction(theta, pos):
    return    pos * theta[2] / theta[1]

In [ ]:
# Plot the samples using columns 1 and 2 of the matrix
fig, ax = plt.subplots(figsize = (8, 8))
colors = ['red', 'green']

data = pd.read_csv('data/logistic_features.csv'); # Load a 3 columns csv file using pandas function

# Each feature is labeled as bias, positive and negative
X = data[['bias', 'positive', 'negative']].values # Get only the numerical values of the dataframe
Y = data['sentiment'].values; # Put in Y the corresponding labels or sentiments

# print(X.shape) # Print the shape of the X part
# print(X) # Print some rows of X

# Color based on the sentiment Y
ax.scatter(np.log(X[:,1]), np.log(X[:,2]), c=[colors[int(k)] for k in Y], s = 0.1)  # Plot a dot for each pair of words
plt.xlabel("Positive (log scale)")
plt.ylabel("Negative (log scale)")

# Now lets represent the logistic regression model in this chart. 
maxpos = np.max(np.log(X[:,1]))

offset = 2 # The pos value for the direction vectors origin

# Plot a gray line that divides the 2 areas.
ax.plot([0,  maxpos], [neg(theta, 0),   neg(theta, maxpos)], color = 'gray') 

# Plot a green line pointing to the positive direction
ax.arrow(offset, neg(theta, offset), offset, direction(theta, offset), head_width=0.25, head_length=0.25, fc='g', ec='g')
# Plot a red line pointing to the negative direction
ax.arrow(offset, neg(theta, offset), -offset, -direction(theta, offset), head_width=0.25, head_length=0.25, fc='r', ec='r')

plt.show()

Para facilitar la interpretación del gráfico hemos representado las características positivas y negativas de cada tuit en escala logarítmica

Del gráfico se desprende que las características que hemos elegido para representar los tuits como vectores numéricos permiten una separación casi perfecta entre tuits positivos y negativos. ¡Así que puede esperar una precisión muy alta para un modelo de clasificación entrenado con estos vectores!

# Entrenamiento de un clasificador con el dataset de Twitter

Una vez analizado cúal es una representación eficiente de la cadenas de texto para entrenar un clasificador de texto (análisis de opinión), vamos a poner esto en práctica, entrenando un clasificador con el dataset de Twiiter, y comprobar si el funcionamiento es tan bueno como cabría esperar

Esta es una función que vamos a utilizar para pre-procesar los tuits. Fíjate que hay algunas diferencias con la función que hemos definido antes. En particular, fíjate que se utiliza una función pos_tag() que extrae la etiqueta morfológica de cada palabra.

Es decir, se utilizan estas etiquetas morfológicas para hacer el pre-procesado de forma más especializada. En concreto, se hace una lematización distinta en función de esta etiqueta. Veremos más sobre análisis morfológico (Part-of Speech Tagging) más adelante. 

Por ahora, vamos a ver qué pinta tiene estas etiquetas morfológicas

In [ ]:
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(pos_tag(tweet_tokens[0]))

Utilizamos esta función para pre-procesar los tuits


In [ ]:
import re, string

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

#print(remove_noise(tweet_tokens[0], stop_words))

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

Y adaptamos la extructura de los datos para el clasificador. El modelo que vamos a utilizr requiere no solo una lista de palabras en un tweet, sino también un diccionario de Python con palabras como claves y True como valores.

In [ ]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

### Ejercicio

Ahora vamos a entrenar el clasificador utilizando un 70% del dataset, y dejando el 30 % restante para validar los resultados. Completa la creacion de los conjuntos de entrenamiento y test (train_data) y (test_data) a partir del conjunto total de datos (dataset)

Asegúrate de que la distribución de tuits positivos y negativos es adecuada. Para ello, fíjate en como se estructura la variable dataset

In [ ]:
import random
from nltk import NaiveBayesClassifier
from nltk import classify

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

# train_data = ???
# test_data = ???

classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))